Dodamo vse importe in definiramo začetno stanje.

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import ast

url_base = "https://urnik.fri.uni-lj.si"

url_year = []
valid_timetables = []

Na url-ju `https://urnik.fri.uni-lj.si/timetable` dobimo seznam vseh urnikov, ki so na voljo. Za nekatere semestre obstaja več urnikov, a ima le en podatke, ostali pa so prazni. Spišemo kodo, ki nam bo poiskala neprazne urnike.

In [ ]:
driver = webdriver.Chrome()
driver.get(url_base + "/timetable")

soup = BeautifulSoup(driver.page_source, 'html.parser')

timetables_section = soup.find('h1', string='Choose timetable').find_parent('div')

# dobimo linke do vseh urnikov
for a_tag in timetables_section.find_all('a'):
    url_year.append(a_tag.get('href'))

# loopamo cez vse urnike
for year in url_year:
    url = url_base + year

    # gremo na zacetno stran urnika za leto in semester v seznamu
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # najdemo sekcijo s skupinami, poiscemo vse programe
    groups_section = soup.find('h2', string='Groups').find_parent('td')
    group_link = groups_section.find_all('a')

    # ce najdemo podatke o skupinah
    if len(group_link) > 0:
        # poiscemo urnik za dodiplomski program vss in preverimo, ali je prazen
        vss = groups_section.find('a', string='1. letnik, Računalništvo in informatika, prva stopnja: visokošolski strokovni')
        vss_url = url_base + vss.get('href')
        
        driver.get(vss_url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # ce ima urnik vsaj en vnos/predmet, ga dodamo na seznam
        subject_link = soup.find(class_="link-subject")
        if subject_link:
            valid_timetables.append(vss_url)


# sortiramo in izpisemo
valid_timetables.sort()
index = 0
for timetable in valid_timetables:
    print(f"{index}. {timetable}")
    index = index + 1

Pogledamo sortiran seznam urnikov. Odstranili bomo leto `2018/19` iz analize, saj letni urnik nima podatkov o skupinah. Imamo 4 urnike za leto `2020/21`, ročno odstranimo duplikate, tako kot urnik `test-predavanja`. Odstranimo še duplikat zimskega semestra leta `2023/24`. Ostane nam seznam validnih urnikov med leti `2019/20` in `2024/25`.<br><br>Odstranimo še zadnji del linka, da nam ostane osnovni link do urnika.

In [ ]:
valid_timetables.pop(16)
valid_timetables.pop(13)
valid_timetables.pop(6)
valid_timetables.pop(5)
valid_timetables.pop(0)

valid_timetables = [url.rpartition('/')[0] for url in valid_timetables]

index = 0
for timetable in valid_timetables:
    print(f"{index}. {timetable}")
    index = index + 1

Tako dobljene linke shranimo pretvorimo v DataFrame objekt in jih shranimo v .csv dokument.

In [ ]:
timetables = pd.DataFrame(valid_timetables, columns=['url'])

def extract_year_semester(url):
    years = url.split('fri-')[1].split('-')[0].split('_')
    year = years[0][2:] + "/" + years[1][2:]

    if 'zimski' in url:
        semester = 1
    else:
        semester = 2

    return year, semester

# Apply the function to the DataFrame
timetables[['url_year', 'url_semester']] = timetables['url'].apply(lambda x: pd.Series(extract_year_semester(x)))

timetables.to_csv('timetables.csv', index=False)

print(timetables)